## Week 3: Clusters and Distributions

In this week's section, we'll explore word representations. Particularly, we'll focus on SVD-based embeddings; you'll have a chance to implement a more sophisticated technique and explore neural embeddings in Assignment 1.

In [23]:
!pip install --upgrade pandas
!pip install --upgrade itertools
!pip install --upgrade collections
!pip install --upgrade vocabulary
!pip install --upgrade scipy
!pip install --upgrade pandas
!pip install --upgrade plotting
!pip install --upgrade bokeh
!pip install --upgrade bhtsne
import os, sys, re, json, time
import itertools
import collections
from IPython.display import display

# NumPy and SciPy for matrix ops
import numpy as np

import scipy.sparse

# Pandas because pandas are awesome
import pandas as pd
# Set pandas floating point display
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

# NLTK for NLP utils
import nltk

import data_utils
reload(data_utils)
import vocabulary
reload(vocabulary)

Requirement already up-to-date: pandas in /usr/local/lib/python2.7/dist-packages
Requirement already up-to-date: pytz>=2011k in /usr/local/lib/python2.7/dist-packages (from pandas)
Requirement already up-to-date: python-dateutil in /usr/local/lib/python2.7/dist-packages (from pandas)
Requirement already up-to-date: numpy>=1.7.0 in /usr/local/lib/python2.7/dist-packages (from pandas)
Requirement already up-to-date: six>=1.5 in /usr/local/lib/python2.7/dist-packages (from python-dateutil->pandas)
/usr/local/lib/python2.7/dist-packages/pip/_vendor/requests/packages/urllib3/util/ssl_.py:318: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#snimissingwarning.

<module 'vocabulary' from 'vocabulary.pyc'>

In [4]:
def flatten(list_of_lists):
    return itertools.chain.from_iterable(list_of_lists)

def pretty_print_matrix(M, rows=None, cols=None, dtype=float):
    display(pd.DataFrame(M, index=rows, columns=cols, dtype=dtype))

# Load corpora

We'll use the `Vocabulary` class again, this time to map words to numerical indices. These will be the row indices of our embedding matrix.

In [5]:
corpus = nltk.corpus.brown
vocab_size = 10000

token_feed = (data_utils.canonicalize_word(w) for w in corpus.words())
vocab = vocabulary.Vocabulary(token_feed, vocab_size)

print "Most common unigrams:"
print "\n".join("\"%s\": %d" % kv 
                for kv in vocab.unigram_counts.most_common(10))

Most common unigrams:
"the": 69971
",": 58334
".": 49346
"of": 36412
"and": 28853
"to": 26158
"a": 23195
"in": 21337
"that": 10594
"is": 10109


## Co-occurrence matrix

We want to build a co-occurrence matrix $C$, where $C_{ij}$ is the number of times word $i$ occurs in the same context as word $j$.

This matrix will be of size `vocab_size` x `vocab_size`, which is quite large - we'd run into the same problem we had with a bigram LM if we represented it explicitly.

Instead, we'll use a **sparse matrix** (`scipy.sparse`). This only stores the non-zero elements, but still supports some of the usual matrix operations like slicing, multiplication, and - most importantly - the SVD. 

See [the scipy.sparse documentation](http://docs.scipy.org/doc/scipy/reference/sparse.html) for more info on how these work; we'll chiefly use the DOK, COO, and CSR formats.

### Fixed-Size Window
It's simplest to implement this for a fixed-size window. Naively, we can just go through the entire corpus, and at each position `i` add a count for the words at positions `i-2, i-1, i+1, i+2`, etc.

Turns out this is really slow in Python, so we'll re-order things: we'll count all pairs `(i,i-2)`, `(i,i-1)`, etc. and add the counts up at the end.

Note that pairs are symmetric: so at the same time we handle `(i,i+k)`, we'll also add an entry for `(i+k,i)`.

In [6]:
def cooccurrence_matrix(ids, vocab, window_size=2):
    # We'll use this as an "accumulator" matrix
    C = scipy.sparse.csc_matrix((vocab.size, vocab.size), 
                                dtype=np.float32)
    
    for k in range(1, window_size+1):
        if k == 0: continue  # don't count (i,i)
        print u"Counting pairs (i, i \u00B1 %d) ..." % k
        # Consider words k positions ahead
        i = ids[:-k] # current word
        j = ids[k:]  # k words ahead
        # Construct a COO matrix: values, indices
        # scipy.sparse will add up duplicates for us
        data = (np.ones_like(i), (i,j))
        Ck_plus = scipy.sparse.coo_matrix(data, shape=C.shape, dtype=np.float32)
        Ck_plus = scipy.sparse.csc_matrix(Ck_plus)
        Ck_minus = Ck_plus.T  # Consider k words behind
        C += Ck_plus + Ck_minus

    print "Co-occurrence matrix: %d words x %d words" % (C.shape)
    print "  %.02g nonzero elements" % (C.nnz)
    return C

Let's look at a toy corpus to see how this works. With a window of 1, we should see co-occurrence counts for each pair of neighboring words:  
`(<s>, nlp)`,  
`(nlp, class)`,  
`(class, is)`,  
and so on - as well as their reversed versions (remember, C is symmetric!)


In [7]:
toy_corpus = [
    "nlp class is awesome",
    "nlp class is fun"
]

words = list(flatten([s.split() for s in toy_corpus]))
toy_vocab = vocabulary.Vocabulary(words)
# sentence_to_ids adds "<s>" and "</s>"
ids = list(flatten(toy_vocab.sentence_to_ids(s.split()) 
                   for s in toy_corpus))

# Here's the important part
toy_C = cooccurrence_matrix(ids, toy_vocab, window_size=1)

toy_labels = toy_vocab.ordered_words()
pretty_print_matrix(toy_C.toarray(), rows=toy_labels, 
                    cols=toy_labels, dtype=int)

Counting pairs (i, i ± 1) ...
Co-occurrence matrix: 8 words x 8 words
  16 nonzero elements


,<s>,</s>,<unk>,nlp,is,class,fun,awesome
<s>,0,1,0,2,0,0,0,0
</s>,1,0,0,0,0,0,1,1
<unk>,0,0,0,0,0,0,0,0
nlp,2,0,0,0,0,2,0,0
is,0,0,0,0,0,2,1,1
class,0,0,0,2,2,0,0,0
fun,0,1,0,0,1,0,0,0
awesome,0,1,0,0,1,0,0,0


In [8]:
# Canonicalize words and convert to numerical IDs
words = data_utils.canonicalize_words(corpus.words(), 
                                      wordset=vocab.word_to_id)
ids = np.array(vocab.words_to_ids(words))

t0 = time.time()
C = cooccurrence_matrix(ids, vocab)
print "Constructed C in %s sec" % data_utils.pretty_timedelta(since=t0)

Counting pairs (i, i ± 1) ...
Counting pairs (i, i ± 2) ...
Co-occurrence matrix: 10000 words x 10000 words
  1e+06 nonzero elements
Constructed C in 0:00:01 sec


### Sentence Context

Naively, we could do this by using something like:
```
for i in sentence_ids[:-1]:
  for j in sentence_ids[1:]:
     C[i,j] += 1
```
But unfortunately, Python is slow, and for long sentences (say, 40 tokens), this can take a loooong time.

However, there's a clever trick. We can compute a matrix $M_{ik}$ where rows $i$ are word indices and columns $k$ are sentence indices. Assuming each word appears only once in a sentence, then we have:

$$ C_{ij} = \sum_{k} \mathbf{1}[w_i \in \text{sentence}\ k] \cdot \mathbf{1}[w_j \in \text{sentence}\ k] = \sum_{k} M_{ik} M_{jk} = (MM^T)_{ij} $$

So we can compute $C_{ij}$ easily with matrix multiplication:

In [9]:
def cooccurrence_matrix_sentences(sentence_ids, vocab):
    M = scipy.sparse.dok_matrix((vocab.size, len(sentence_ids)), 
                                dtype=np.int32)
    
    for j,ids in enumerate(sentence_ids):
        for i in ids:
            M[i,j] += 1
    
    M = scipy.sparse.csr_matrix(M)
    # Correction for multiple occurrences of words
    diag_corr = scipy.sparse.dia_matrix((np.ravel(M.sum(1)), [0]),
                                        shape=(vocab.size, vocab.size))
    C = M.dot(M.T) - diag_corr
    print "Co-occurrence matrix: %d words x %d words" % (C.shape)
    print "  %.02g nonzero elements" % (C.nnz)
    return C
    
sentence_ids = [vocab.words_to_ids(data_utils.canonicalize_words(s)) 
                for s in corpus.sents()]
t0 = time.time()
C_sentence = cooccurrence_matrix_sentences(sentence_ids, vocab)
print "Constructed C in %s" % data_utils.pretty_timedelta(since=t0)

Co-occurrence matrix: 10000 words x 10000 words
  4.7e+06 nonzero elements
Constructed C in 0:00:39


**Note:** It's also common to do SVD directly on the word-sentence matrix M, particularly where instead of sentences we use paragraphs or whole documents. This is known as Latent Semantic Analysis, or LSA.

Let's try this on our toy corpus again. You should notice a somewhat denser matrix this time: every pair of words are represented, not just the adjacent ones.

In [10]:
sentence_ids = [toy_vocab.sentence_to_ids(s.split()) 
                for s in toy_corpus]

toy_C_s = cooccurrence_matrix_sentences(sentence_ids, toy_vocab)

pretty_print_matrix(toy_C_s.toarray(), rows=toy_labels, 
                    cols=toy_labels, dtype=int)

Co-occurrence matrix: 8 words x 8 words
  40 nonzero elements


,<s>,</s>,<unk>,nlp,is,class,fun,awesome
<s>,0,2,0,2,2,2,1,1
</s>,2,0,0,2,2,2,1,1
<unk>,0,0,0,0,0,0,0,0
nlp,2,2,0,0,2,2,1,1
is,2,2,0,2,0,2,1,1
class,2,2,0,2,2,0,1,1
fun,1,1,0,1,1,1,0,0
awesome,1,1,0,1,1,1,0,0


## Computing Word Vectors

In order to go from our co-occurrence matrix to word vectors, we need to do two things:

- First, convert to **PPMI** to reduce the impact of common words.
- Compute the SVD, and extract our vectors.

PPMI stands for Positive [Pointwise Mutual Information](https://en.wikipedia.org/wiki/Pointwise_mutual_information). PMI is a generalization of the idea of correlation, but for arbitrary variables. Here, we're interested in the correlation between word $i$ and word $j$, where we take the samples to be all the word-word pairs in our corpus.  
Positive just means we'll truncate at zero: $\text{PPMI}(i,j) = \max(0, \text{PMI}(i,j))$

We'll apply PPMI as a transformation of our counts matrix. First, compute probabilities:
$$ P(i,j) = \frac{C(i,j)}{\sum_{k,l} C(k,l)} = \frac{C_{ij}}{Z}$$
$$ P(i) = \frac{\sum_{k} C(i,k)}{\sum_{k,l} C(k,l)} = \frac{Z_i}{Z}$$

Then compute PPMI:
$$ \text{PMI}(i,j) = \log \frac{P(i,j)}{P(i)P(j)} = \log \frac{C_{ij} \cdot Z}{Z_i \cdot Z_j} $$
$$\text{PPMI}(i,j) = \max(0, \text{PMI}(i,j))$$


In [11]:
def PPMI(C):
    # Expect C to be a CSC matrix
    Z = float(C.sum())  # total counts
    Zc = np.array(C.sum(axis=0), dtype=np.float64).flatten() # sum each column (along rows)
    Zr = np.array(C.sum(axis=1), dtype=np.float64).flatten() # sum each row (along columns)
    
    # Get indices of relevant elements
    ii, jj = C.nonzero()  # row, column indices
    Cij = np.array(C[ii,jj], dtype=np.float64).flatten()
    pmi = np.log(Cij * Z / (Zr[ii] * Zc[jj]))
    ppmi = np.maximum(0, pmi)  # take positive only
    ret = scipy.sparse.csc_matrix((ppmi, (ii,jj)), shape=C.shape,
                                  dtype=np.float64)
    ret.eliminate_zeros()  # remove zeros
    return ret

Now to compute the SVD, we can just use [`sklearn.decomposition.TruncatedSVD`](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html):

In [12]:
from sklearn.decomposition import TruncatedSVD
def SVD(X, k=100):
    transformer = TruncatedSVD(n_components=k, random_state=1)
    Wv = transformer.fit_transform(X)
    # Normalize to unit length
    Wv = Wv / np.linalg.norm(Wv, axis=1).reshape([-1,1])
    return Wv, transformer.explained_variance_

In [13]:
t0 = time.time()
X = PPMI(C)
print "Computed PPMI in %s" % data_utils.pretty_timedelta(since=t0)

t0 = time.time()
Wv, svs = SVD(X)
print "Computed SVD in %s" % data_utils.pretty_timedelta(since=t0)

Computed PPMI in 0:00:00
Computed SVD in 0:00:03


Let's see what this does on our toy corpus:

In [14]:
pretty_print_matrix(PPMI(toy_C).toarray(), rows=toy_labels, 
                    cols=toy_labels, dtype=float)

,<s>,</s>,<unk>,nlp,is,class,fun,awesome
<s>,0.0000,0.8938,0.0000,1.2993,0.0000,0.0000,0.0000,0.0000
</s>,0.8938,0.0000,0.0000,0.0000,0.0000,0.0000,1.2993,1.2993
<unk>,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
nlp,1.2993,0.0000,0.0000,0.0000,0.0000,1.0116,0.0000,0.0000
is,0.0000,0.0000,0.0000,0.0000,0.0000,1.0116,1.0116,1.0116
class,0.0000,0.0000,0.0000,1.0116,1.0116,0.0000,0.0000,0.0000
fun,0.0000,1.2993,0.0000,0.0000,1.0116,0.0000,0.0000,0.0000
awesome,0.0000,1.2993,0.0000,0.0000,1.0116,0.0000,0.0000,0.0000


And if we take the SVD, we'll get the words represented by row vectors. Note that "fun" and "awesome" get the same representation, since they're interchangable in our (tiny) corpus. How do their embedding vectors relate to the one for "class"?

In [15]:
k = 3
pretty_print_matrix(SVD(PPMI(toy_C), k=k)[0], rows=toy_labels, 
                    cols=range(k), dtype=float)

,0,1,2
<s>,0.7576,0.0000,0.6527
</s>,-0.0000,1.0000,0.0000
<unk>,0.0536,0.1367,0.9892
nlp,0.0000,1.0000,-0.0000
is,0.0000,1.0000,-0.0000
class,0.7800,0.0000,0.6258
fun,0.9373,0.0000,-0.3484
awesome,0.9373,0.0000,-0.3484


# Visualize Word Vectors

We've included some code that will generate an interactive visualization of the word cloud. You can use the mouse to pan and zoom, and you can pass a dict of colors to highlight or color-code words.

It's implemented using the [Bokeh](http://bokeh.pydata.org/en/latest/) library, which is similar to Plotly but much faster at rendering text; check out `plotting.py` (in this directory) if you're interested.

In [18]:
import plotting as plot_wv
reload(plot_wv)
import bokeh.plotting as bp
bp.output_notebook()

Loading BokehJS ...

In [19]:
# Plot the top 1000 words
plot_wv.plot_wv(Wv, vocab, num_words=1000)

In [20]:
# Plot all the word vectors in a separate window.
# The page might load slowly on older machines.
plot_wv.plot_wv(Wv, vocab, num_words=vocab_size, inline=False, 
                filename="plots/wordvectors.html")

We can also highlight specific words by passing a color dict:

In [21]:
word_colors = collections.defaultdict(lambda: "black")
word_colors["the"] = "blue"
word_colors["england"] = "red"
word_colors["soviet"] = "red"
plot_wv.plot_wv(Wv, vocab, num_words=1000, word_colors=word_colors)

## t-SNE

The above visualization just plots the first two dimensions. This is equivalent to doing a TruncatedSVD with k=2, which doesn't always give the most meaningful representation.

We can get more intuition by projecting down with [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding). This is a *non*-linear way of embedding high-dimensional data (like our embedding vectors) into a low dimensional space. It works by preserving local distances (like nearby neighbors), at the expense of some global distortion.

t-SNE won't be very good if we want to check analogy relationships, but it will help us visualize clusters.

Scikit-learn includes a t-SNE implementation in [`sklearn.manifold.TSNE`](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html), but the implementation is slow and tends to crash by using too much (>4 GB) memory.

Instead, we'll use the excellent [`bhtsne`](https://github.com/dominiek/python-bhtsne) package. Install with:
```
sudo apt-get install gcc g++
pip install bhtsne
```

The cell below will take around 2 minutes to run on a 2 CPU Cloud Compute instance.

In [26]:
!pip install --upgrade bhtsne
import bhtsne

t0 = time.time()
print "Running Barnes-Hut t-SNE on word vectors; matrix shape = %s" % str(Wv.shape)
Wv2 = bhtsne.tsne(Wv)
print "Transformed in %s" % data_utils.pretty_timedelta(since=t0)

## Uncomment below if you need to use sklearn implementation
## (not recommended)
# from sklearn.manifold import TSNE
# transformer = TSNE(n_components=2, verbose=2)
# Wv2 = transformer.fit_transform(Wv)

  Using cached bhtsne-0.1.9.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-build-9aYIHE/bhtsne/setup.py", line 4, in <module>
        from Cython.Build import cythonize
    ImportError: No module named Cython.Build
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-build-9aYIHE/bhtsne/


ImportError: No module named bhtsne

In [27]:
plot_wv.plot_wv(Wv2, vocab, num_words=1000)

NameError: name 'Wv2' is not defined

In [28]:
plot_wv.plot_wv(Wv2, vocab, num_words=vocab_size, inline=False, 
                filename="plots/wordvectors_tsne.html")

NameError: name 'Wv2' is not defined

# Exploration

Use the cells below to experiment with word embeddings. Things to try:

- Experiment with different window sizes. How do the word clusters change if you use a window of $\pm 1$ word, versus $\pm 3$, or full-sentence?
- Look at a few "target" words of interest, using `word_colors` to highlight, or the `show_nns` function below. What are their nearest neighbors, and how does this change with the way you construct the embeddings?

Feel free to modify any of the code below, or to write your own!

In [29]:
def find_nn_cos(word_id, Wv, k=10):
    """Find nearest neighbors, by cosine distance."""
    v = Wv[word_id]
    Z = np.linalg.norm(Wv, axis=1) * np.linalg.norm(v)
    ds = np.dot(Wv, v.T) / Z
    nns = np.argsort(-1*ds)[:k]  # sort descending, take best
    return nns, ds[nns]  # word indices, distances

def show_nns(word, Wv, vocab, k=10):
    print "Nearest neighbors for \"%s\"" % word
    for i, d in zip(*find_nn_cos(vocab.word_to_id[word], Wv, k)):
        w = vocab.id_to_word[i]
        print "%.03f : \"%s\"" % (d, w)

In [30]:
# Input lists
sentence_ids = [vocab.words_to_ids(data_utils.canonicalize_words(s)) 
                for s in corpus.sents()]
ids = list(flatten(sentence_ids))

# Compute co-occurence matrix and word vectors
C = cooccurrence_matrix(ids, vocab, window_size=1)
# C = cooccurrence_matrix_sentences(sentence_ids, vocab)

Wv, _ = SVD(PPMI(C), k=100)

Wv2 = bhtsne.tsne(Wv)

Counting pairs (i, i ± 1) ...
Co-occurrence matrix: 10000 words x 10000 words
  5.5e+05 nonzero elements


NameError: name 'bhtsne' is not defined

In [31]:
show_nns("close", Wv, vocab)

Nearest neighbors for "close"
1.000 : "close"
0.698 : "hard"
0.646 : "fast"
0.633 : "near"
0.591 : "quietly"
0.588 : "soon"
0.581 : "easy"
0.578 : "quickly"
0.575 : "hot"
0.574 : "bad"
